# Imports

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime as dt
from pathlib import Path
from pytz import timezone
from hbt_tools import utils_hbt as hu
from hbt_tools import plots as hp

# Functions

In [ ]:
def filter_ts(ts, date_from, date_to):
    if not date_to:
        ts = ts[(ts.index>=date_from)]
    elif not date_from:
        ts = ts[(ts.index<date_to)]
    else:
        ts = ts[(ts.index>=date_from) & (ts.index<date_to)]

    return ts

# Parameters

In [ ]:
save_dir = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Datenauswertung\Plots")

mes_dict = {
    'Zulauf Splittfilterbecken': {
        'id': 	1078,
        'units': 'cm',
        'yaxis': 'y1',
    },

    'Trübung Zulauf': {
        'id': 	1077,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Trübung Ablauf': {
        'id': 	1081,
        'units': 'NTU',
        'yaxis': 'y2',
    },

    'Niveau Retentionssickerbecken': {
        'id': 	1082,
        'units': 'cm',
        'yaxis': 'y3',
    }
}

# Read Rain

In [ ]:

path_rain = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Regendaten\therwil_2207-2307.csv")
date_from = dt(2022, 7, 29, 8, tzinfo=timezone('Europe/Zurich'))

rain = hu.read_raindata_agrometeo(path_rain, resolution='10min')
rain = filter_ts(rain, date_from=date_from, date_to=None)
rain['kumulativ'] = rain.cumsum()

# Read Data

In [ ]:
for k, v in mes_dict.items():
    data = hu.hbtdb_get_data(v['id'])
    mes_dict[k].update({'data': data})

# Overview Plot

In [ ]:
fig = go.Figure()

for k, v in mes_dict.items():
    if v['yaxis']:
        yaxis = v['yaxis']
    
    else:
        yaxis = None

    fig.add_trace(go.Scatter(
        x=v['data'].index,
        y=v['data'].iloc[:,0],
        name=k,
        yaxis=yaxis
    ))

fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,0],
    name='Regen',
    yaxis='y4',
    line=dict(color='blue')
))

In [ ]:
ylabels = ['Durchfluss [l/s]', 'Trübung [NTU]', 'Niveau [cm]', 'Regen [mm]']

# Create axis objects
props_dict = {}
props_dict.update({
    'xaxis': {
        'domain': [0.1 + 0.01, 1.0]
    }
})

for i, ylabel in enumerate(ylabels):
    yaxis = f'yaxis{i+1}'
    if i==0:
        props_dict.update({yaxis: {}})
        props_dict[yaxis].update({
            'title': ylabel,
            'side': 'left',
            'anchor':'x'
        })

    else:
        props_dict.update({yaxis: {}})
        props_dict[yaxis].update({
            'title': ylabel,
            'side': 'left',
            'overlaying': 'y',
            'autoshift': True,
            'anchor':'free'
        })

props_dict['yaxis1'].update({'anchor': 'x'})

In [ ]:
fig.update_layout(props_dict)

In [ ]:
save_path = save_dir / '230804_Übersicht-Messungen.html'
fig.write_html(save_path)

# Zulauf

In [ ]:
id = 1078
q_zu = hu.hbtdb_get_data(id)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=q_zu.index,
    y=q_zu.iloc[:,0],
    name='Zulauf Splittfilterbecken',
    mode='lines+markers',
    line=dict(color='grey', width=1),
    marker=dict(color='cyan', size=4)
));

fig.add_trace(go.Scatter(
    x=rain.index,
    y=rain.iloc[:,1],
    name='Regen kumulativ',
    yaxis='y2',
    line=dict(dash='dash', color='blue')
));

In [ ]:
fig.update_layout(
    xaxis={
        'domain': [0.1, 1]
    },
    yaxis1={
        'title': 'Durchfluss [l/s]',
        'anchor':'x'
    },
    yaxis2={
        'title': 'Regen [mm]',
        'overlaying': 'y',
        'anchor':'free',
        'autoshift': True,
    }
);

In [ ]:
fig.update_layout(
    title='Zulauf Splittfilterbecken', font=dict(size=25)
)

fig

In [ ]:
save_path = save_dir / 'Zulauf-Splittfilterbecken.html'
fig.write_html(save_path)

# Niveau Becken

In [ ]:
# Paths
path_10min = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau"
    r"\05 Berechnungen Grundlagen\Datenauswertung"
    r"\ARA Birs_SABA Filter_Aug-Okt 2022.csv")
path_1h = Path(r"Q:\Projekte\8000-\8500er\8596\8596.35-Hagnau\05 Berechnungen"
    r" Grundlagen\Datenauswertung\SABA Daten.csv")

# Read data
dt_format = '%d.%m.%Y %H:%M'
date_parser = lambda x: pd.to_datetime(x, format=dt_format)

data_10min = pd.read_csv(path_10min, index_col=0, parse_dates=True, sep=';',
    date_parser=date_parser)
data_1h = pd.read_csv(path_1h, parse_dates=True, date_parser=date_parser,
    skiprows=5, header=None, index_col=0, sep=';')

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_1h.index, y=data_1h.iloc[:,0], name='1 Stunde'))
fig.add_trace(go.Scatter(x=data_10min.index, y=data_10min.iloc[:,0],
    name='10 Minuten'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Niveau [m]')

save_path = save_dir / 'Niveau_Auflösung_1h_10min.html'
fig.write_html(save_path)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data_1h.index, y=data_1h.iloc[:,0],
    name='B2.XA2.QA.23'))
fig.add_trace(go.Scatter(x=data_1h.index, y=data_1h.iloc[:,1],
    name='B2.XA2.QA.20'))

fig.update_xaxes(title='Datum')
fig.update_yaxes(title='Niveau [m]')

save_path = save_dir / 'Niveau_Filter_Splittfilter.html'
fig.write_html(save_path)